In [1]:
from pandas_datareader import data
import pandas as pd
from datetime import datetime
import yfinance as yf

In [2]:
securities = {
    'AAPL':'AAPL',
    'SSEC':'000001.SS',
    'DJI':'^DJI',
    'SZC':'399001.SZ'
}

In [3]:
def YahooReader(security,start_date, end_date):
    df = yf.download(securities[security], start_date, end_date)
    df['dif'] = df.Close.diff()
    df['Date'] = df.index
    df = df.fillna(0)
    return df 
df = YahooReader('SZC','2021-01-13', '2021-01-15')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,dif,Date
Date,,,,,,,,
2021-01-13,15491.070312,15597.969727,15240.900391,15365.429688,15365.429688,2193700,0.000000,2021-01-13
2021-01-14,15283.559570,15354.120117,15033.469727,15070.129883,15070.129883,2011400,-295.299805,2021-01-14


In [4]:
start_date = '2016-01-01'
end_date = '2021-01-14'

from pandas_datareader import data
def YahooReader(security,start_date, end_date):
    df = data.DataReader(securities[security],'yahoo', start_date, end_date)
    df['dif'] = df.Close.diff()
    df['Date'] = df.index
    df = df.fillna(0)
    return df 
df = YahooReader('SSEC','2021-01-13', '2021-01-15')
df

RemoteDataError: Unable to read URL: https://finance.yahoo.com/quote/000001.SS/history?period1=1610481600&period2=1610740799&interval=1d&frequency=1d&filter=history
Response Text:
b'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n

In [4]:
import sqlite3 as sql

In [5]:
conn = sql.connect('stock')

In [6]:
def make_stocktable(con,table_name):
    make_table_sql = """
    CREATE TABLE %s (
        High Double,
        Low Double,
        Open Double,
        Close Double,
        Volume Double,
        Adj_Close Double,
        dif Double,
        Date TIMESTAMP PRIMARY KEY
    )
    """%table_name

    con.execute(make_table_sql)
    con.commit()

In [7]:
def drop_table(con,table_name):
    con.execute('Drop table {}'.format(table_name))
    con.commit()
drop_table(conn, 'DJI')

In [8]:
def insert_stocktable(df,con,table_name):
    for i in range(len(df)):
        dates = pd.read_sql_query("SELECT Date from %s"%table_name, con)
        con.commit()
        values = [table_name]+list(df.iloc[i,:-1])+["'"+str(df.iloc[i,-1])+"'"]
        if values[-1] not in dates:
            sql_script = "INSERT INTO {}(High,Low,Open,Close,Volume,Adj_Close,dif,Date) VALUES ({},{},{},{},{},{},{},{})".format(*values)
            try:
                con.execute(sql_script)
            except:
                pass
            con.commit()

In [40]:
data= pd.read_sql_query("SELECT * from SSEC", conn)
data['Date'] = pd.to_datetime(data.Date)
data.index = data.Date
data

,High,Low,Open,Close,Volume,Adj_Close,dif,Date
Date,,,,,,,,
2016-01-04,3538.688965,3295.740967,3536.589111,3296.258057,184400.0,3296.258057,0.000000,2016-01-04
2016-01-05,3328.138916,3189.604980,3196.650879,3287.710938,266900.0,3287.710938,-8.547119,2016-01-05
2016-01-06,3362.974121,3288.933105,3291.195068,3361.840088,238900.0,3361.840088,74.129150,2016-01-06
2016-01-07,3309.656982,3115.885010,3309.656982,3125.001953,70600.0,3125.001953,-236.838135,2016-01-07
2016-01-08,3235.450928,3056.877930,3194.625000,3186.412109,286400.0,3186.412109,61.410156,2016-01-08
...,...,...,...,...,...,...,...,...
2020-01-06,3107.202881,3065.309082,3070.908936,3083.407959,312600.0,3083.407959,-0.377930,2020-01-06
2020-01-07,3105.450928,3084.329102,3085.488037,3104.802002,276600.0,3104.802002,21.394043,2020-01-07
2020-01-08,3094.239014,3059.131104,3094.239014,3066.893066,297900.0,3066.893066,-37.908936,2020-01-08


In [13]:
a = data.index[0]
a.date()

datetime.date(2016, 1, 4)

In [14]:
a = '2015-12-31 00:00:00'

In [15]:
import datetime
today = pd.Timestamp(datetime.datetime.today().date())
today > df.Date[-2]

True

In [16]:
data.index[-1]

Timestamp('2021-01-14 00:00:00')

In [17]:
pd.to_datetime(data.Date)[0]

Timestamp('2016-01-04 00:00:00')

In [43]:
def update_data(security):
    con = sql.connect('stock')
    dates = pd.read_sql_query("SELECT Date from %s"%security, con)
    last_date = pd.Timestamp(dates.values[-1][0])
    today = pd.Timestamp(datetime.today().date())
    if today > last_date:
        new_data = YahooReader(security,last_date,today)
        insert_stocktable(new_data.iloc[1:,],con,security)

    df = pd.read_sql_query("SELECT * from %s"%security, con)
    df.index = df.Date
    con.close()
    return df

a = update_data('SSEC')
a

,High,Low,Open,Close,Volume,Adj_Close,dif,Date
Date,,,,,,,,
2016-01-04 00:00:00,3538.688965,3295.740967,3536.589111,3296.258057,1.844000e+05,3296.258057,0.000000,2016-01-04 00:00:00
2016-01-05 00:00:00,3328.138916,3189.604980,3196.650879,3287.710938,2.669000e+05,3287.710938,-8.547119,2016-01-05 00:00:00
2016-01-06 00:00:00,3362.974121,3288.933105,3291.195068,3361.840088,2.389000e+05,3361.840088,74.129150,2016-01-06 00:00:00
2016-01-07 00:00:00,3309.656982,3115.885010,3309.656982,3125.001953,7.060000e+04,3125.001953,-236.838135,2016-01-07 00:00:00
2016-01-08 00:00:00,3235.450928,3056.877930,3194.625000,3186.412109,2.864000e+05,3186.412109,61.410156,2016-01-08 00:00:00
...,...,...,...,...,...,...,...,...
2021-01-08 00:00:00,3588.062988,3544.891113,3577.691895,3570.107910,3.456000e+05,3570.107910,-6.097168,2021-01-08 00:00:00
2021-01-11 00:00:00,3597.701904,3516.986084,3571.321045,3531.498047,3.625000e+05,3531.498047,-38.609863,2021-01-11 00:00:00
2021-01-12 00:00:00,3608.340088,3517.469971,3518.010010,3608.340088,3.234000e+05,3608.340088,76.842041,2021-01-12 00:00:00


In [25]:
def add_security(security):
    today =  pd.Timestamp(datetime.today().date())
    df = YahooReader(security,'2016-01-01',today)
    con = sql.connect('stock')
    make_stocktable(con,security)
    insert_stocktable(df,con,security)
    con.close()
add_security("DJI")

In [39]:
def renew_security(security,start_date):
    today =  pd.Timestamp(datetime.today().date())
    df = YahooReader(security,start_date,today)
    con = sql.connect('stock')
    con.execute("DELETE FROM {} WHERE Date >= '{}'".format(security,start_date))
    #insert_stocktable(df,con,security)
    con.commit()
    con.close()
renew_security('SSEC','2020-01-12')    